# Machine Learning Pipelines  

### Column Transformer

Often times, you may not want to simply apply every function to all columns. If our columns are of different types, we may only want to apply certain parts of the pipeline to a subset of columns. This is what we saw in the two previous exercises. One set of transformations are applied to numeric columns and another set to the categorical ones. We can use scikit-learn‘s ColumnTransformer as one way of combining these processes together.

ColumnTransformer takes in a list of tuples of the form (name, pipeline, columns):

example_column_transformer = ColumnTransformer(
    transformers=[ ("name_1", pipeline_1, columns_1),
                   ("name_2", pipeline_2, columns_2)])

The transformer can be anything with a .fit and .transform method like we used previously (like SimpleImputer or StandardScaler), but can also itself be a pipeline.

## Score method of the pipeline object

 But now the final step also has a .predict method, which can be called on the entire pipeline! Additionally the .score() method, which estimates the default prediction score on any scikit-learn model can also be used to evaluate the performance of the pipeline.

 pipeline_score = pipeline.score(x_test,y_test)

## Hyperparameter Tuning

Great, we have a very condensed bit of code that does all our data cleaning, preprocessing, and modeling in a reusable fashion! What now? Well, we can tune some of the parameters of the model by applying a grid search over a range of hyperparameter values.

A linear regression model has very few hyperparameters and here we’ll be using the hyperparameter that pertains to whether we include an intercept or not. 